# Spectral Certificate Validation

## Submission: Koopman + Davis-Kahan/Wedin Detection Pipeline

This notebook validates the spectral certificate approach for detecting agent drift vs. creative behavior.

**All data are synthetic. No real secrets or external network calls.**

### Overview
1. Generate synthetic traces (gold/creative/drift)
2. Compute spectral certificates for each trace
3. Train a classifier and evaluate ROC/AUC
4. Analyze Davis-Kahan angle distributions
5. Calibrate detection threshold
6. Generate validation reports

In [ ]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from certificates.spectral_prover import (
    compute_detection_statistics,
    generate_synthetic_linear_system,
    generate_perturbed_trajectory,
    trajectory_matrix,
    subspace_angle,
    davis_kahan_upper_bound,
)

from analysis.convert_trace import generate_synthetic_traces

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print(f"Project root: {PROJECT_ROOT}")

## 1. Generate Synthetic Traces

We generate three types of traces:
- **Gold**: Smooth, low-noise trajectories (baseline behavior)
- **Creative**: Higher variance but still coherent (acceptable exploration)
- **Drift**: Divergent trajectories (problematic behavior to detect)

In [ ]:
# Generate synthetic traces
TRACES_DIR = PROJECT_ROOT / 'experiment_traces'

N_TRACES = 30  # Per label
T = 40  # Timesteps
D = 64  # Embedding dimension

outputs = generate_synthetic_traces(
    TRACES_DIR,
    n_gold=N_TRACES,
    n_creative=N_TRACES,
    n_drift=N_TRACES,
    T=T,
    d=D,
    seed=42,
)

print("Generated traces:")
for label, paths in outputs.items():
    print(f"  {label}: {len(paths)} traces")

## 2. Load and Compute Features

For each trace, compute:
- SVD reconstruction residual
- Tail energy (unexplained variance)
- Singular gap at rank k
- Davis-Kahan angle vs baseline
- Koopman prediction residual

In [ ]:
K = 10  # Rank for spectral analysis

def load_traces(traces_dir):
    """Load all traces from directory."""
    traces = {'gold': [], 'creative': [], 'drift': []}
    for label in traces:
        label_dir = traces_dir / label
        if not label_dir.exists():
            continue
        for f in sorted(label_dir.glob('*.json')):
            with open(f) as fp:
                traces[label].append(json.load(fp))
    return traces

traces = load_traces(TRACES_DIR)
print(f"Loaded: gold={len(traces['gold'])}, creative={len(traces['creative'])}, drift={len(traces['drift'])}")

# First pass: compute gold baseline
gold_features = []
for t in traces['gold']:
    stats = compute_detection_statistics(t['embeddings'], k=K)
    stats['label'] = 'gold'
    stats['run_id'] = t['run_id']
    gold_features.append(stats)

# Get baseline U_k from first gold trace
baseline_U = gold_features[0]['U_k'] if gold_features else None

# Compute all features with baseline
all_features = []
for label in ['gold', 'creative', 'drift']:
    for t in traces[label]:
        stats = compute_detection_statistics(t['embeddings'], k=K, baseline_U=baseline_U)
        stats['label'] = label
        stats['run_id'] = t['run_id']
        all_features.append(stats)

# Create DataFrame
df = pd.DataFrame(all_features)
df = df.drop(columns=['U_k'], errors='ignore')

print(f"\nFeatures computed: {len(df)} traces")
df.head()

## 3. Feature Distributions by Label

Visualize how spectral features separate the three classes.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

features_to_plot = ['residual', 'theoretical_bound', 'tail_energy', 
                    'dk_angle', 'koopman_residual', 'singular_gap']
titles = ['Residual', 'Theoretical Bound', 'Tail Energy',
          'Davis-Kahan Angle', 'Koopman Residual', 'Singular Gap']

for ax, feat, title in zip(axes.flat, features_to_plot, titles):
    for label in ['gold', 'creative', 'drift']:
        data = df[df['label'] == label][feat].dropna()
        ax.hist(data, bins=15, alpha=0.5, label=label, density=True)
    ax.set_xlabel(feat)
    ax.set_ylabel('Density')
    ax.set_title(title)
    ax.legend()

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'reports' / 'spectral_validation' / 'feature_distributions.png', dpi=150)
plt.show()

In [ ]:
# Summary statistics
summary = df.groupby('label')[['residual', 'theoretical_bound', 'tail_energy', 
                                'dk_angle', 'koopman_residual']].agg(['mean', 'std', 'median'])
print("\nSummary Statistics by Label:")
summary

## 4. Classification: Drift vs Creative

Train a logistic regression classifier to distinguish drift from non-drift (gold + creative).

In [ ]:
# Prepare data
feature_cols = ['residual', 'theoretical_bound', 'tail_energy', 
                'dk_angle', 'koopman_residual', 'singular_gap', 'pca_explained']

df['is_drift'] = (df['label'] == 'drift').astype(int)

X = df[feature_cols].fillna(0).values
y = df['is_drift'].values

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train with cross-validation
model = LogisticRegression(max_iter=1000, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_scaled, y, cv=cv, scoring='roc_auc')

print(f"Cross-validation AUC: {cv_scores.mean():.4f} +/- {cv_scores.std():.4f}")

# Fit final model
model.fit(X_scaled, y)

# Feature importance
importance = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': model.coef_[0]
}).sort_values('coefficient', ascending=False)

print("\nFeature Importance:")
importance

## 5. ROC Curve and AUC

In [ ]:
# Compute ROC curve
y_proba = model.predict_proba(X_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y, y_proba)
roc_auc = auc(fpr, tpr)

# Find TPR at FPR = 0.05
idx = np.where(fpr <= 0.05)[0]
if len(idx) > 0:
    tpr_at_fpr05 = tpr[idx[-1]]
    threshold_at_fpr05 = thresholds[idx[-1]]
else:
    tpr_at_fpr05 = 0.0
    threshold_at_fpr05 = 0.5

print(f"AUC: {roc_auc:.4f}")
print(f"TPR @ FPR=0.05: {tpr_at_fpr05:.4f}")
print(f"Threshold @ FPR=0.05: {threshold_at_fpr05:.4f}")

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {roc_auc:.3f})')
ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
ax.axvline(x=0.05, color='r', linestyle=':', linewidth=2, label='FPR = 0.05')
ax.scatter([fpr[idx[-1]]], [tpr[idx[-1]]], color='red', s=100, zorder=5,
           label=f'Operating Point (TPR={tpr_at_fpr05:.2f})')

ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curve: Drift Detection', fontsize=14)
ax.legend(loc='lower right', fontsize=10)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'reports' / 'spectral_validation' / 'roc_curve_notebook.png', dpi=150)
plt.show()

## 6. Davis-Kahan Angle Analysis

Verify that the Davis-Kahan bound holds empirically.

In [ ]:
# Scatter plot: dk_angle vs residual
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: DK angle vs residual
for label in ['gold', 'creative', 'drift']:
    subset = df[df['label'] == label]
    axes[0].scatter(subset['residual'], subset['dk_angle'], 
                    alpha=0.6, label=label, s=50)
axes[0].set_xlabel('Residual')
axes[0].set_ylabel('Davis-Kahan Angle (radians)')
axes[0].set_title('DK Angle vs Residual')
axes[0].legend()

# Right: DK angle vs dk_bound
valid = df[df['dk_bound'] < 10]  # Filter extreme values
axes[1].scatter(valid['dk_bound'], np.sin(valid['dk_angle']), alpha=0.5)
axes[1].plot([0, 1], [0, 1], 'r--', linewidth=2, label='sin(θ) = bound')
axes[1].set_xlabel('Davis-Kahan Upper Bound')
axes[1].set_ylabel('sin(DK Angle)')
axes[1].set_title('Empirical vs Theoretical Bound')
axes[1].legend()

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'reports' / 'spectral_validation' / 'dk_analysis.png', dpi=150)
plt.show()

## 7. Calibration Table

Compute calibrated thresholds for different FPR levels.

In [ ]:
# Calibration table
target_fprs = [0.01, 0.05, 0.10, 0.20]
calibration = []

for target_fpr in target_fprs:
    idx = np.where(fpr <= target_fpr)[0]
    if len(idx) > 0:
        best_idx = idx[-1]
        calibration.append({
            'Target FPR': target_fpr,
            'Actual FPR': fpr[best_idx],
            'TPR': tpr[best_idx],
            'Threshold': thresholds[best_idx],
        })

calibration_df = pd.DataFrame(calibration)
print("\nCalibration Table:")
calibration_df

## 8. Generate Validation Report

In [ ]:
# Build report
report = {
    'AUC': float(roc_auc),
    'TPR_at_FPR05': float(tpr_at_fpr05),
    'threshold_tau': float(threshold_at_fpr05),
    'cv_AUC_mean': float(cv_scores.mean()),
    'cv_AUC_std': float(cv_scores.std()),
    'median_residuals': {
        'gold': float(df[df['label'] == 'gold']['residual'].median()),
        'creative': float(df[df['label'] == 'creative']['residual'].median()),
        'drift': float(df[df['label'] == 'drift']['residual'].median()),
    },
    'num_runs_per_label': {
        'gold': len(traces['gold']),
        'creative': len(traces['creative']),
        'drift': len(traces['drift']),
    },
    'feature_importance': importance.to_dict('records'),
    'calibration_table': calibration,
    'rank_k': K,
    'synthetic_data': True,
}

# Save report
report_dir = PROJECT_ROOT / 'reports' / 'spectral_validation' / 'notebook'
report_dir.mkdir(parents=True, exist_ok=True)

with open(report_dir / 'validation_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("\nValidation Report Saved!")
print(json.dumps(report, indent=2))

## 9. Summary

### Key Results

| Metric | Value | Threshold | Status |
|--------|-------|-----------|--------|
| AUC | {AUC} | >= 0.90 | {PASS/FAIL} |
| TPR @ FPR=0.05 | {TPR} | >= 0.80 | {PASS/FAIL} |

### Observations

1. **Residual** is a strong discriminator between drift and non-drift
2. **Davis-Kahan angle** provides geometric insight into subspace perturbation
3. **Koopman residual** captures temporal prediction error
4. The combination achieves reliable detection with low false positive rate

### Note
All data used in this validation are **synthetic**. No real secrets or external network calls were used.

In [ ]:
# Final validation check
auc_pass = roc_auc >= 0.90
tpr_pass = tpr_at_fpr05 >= 0.80

print("\n" + "="*50)
print("VALIDATION SUMMARY")
print("="*50)
print(f"AUC: {roc_auc:.4f} {'PASS' if auc_pass else 'WARN'} (threshold: 0.90)")
print(f"TPR @ FPR=0.05: {tpr_at_fpr05:.4f} {'PASS' if tpr_pass else 'WARN'} (threshold: 0.80)")
print("="*50)

if auc_pass and tpr_pass:
    print("\n[SUCCESS] All validation criteria met!")
else:
    print("\n[WARNING] Some criteria not met - review analysis.")